In [1]:
import numpy as np
import tensorflow as tf
import pylab
%matplotlib inline

from tensorflow.python.framework import ops
from tensorflow.python.ops import gen_nn_ops

In [2]:
X_data = np.loadtxt('./coordinates1.csv', delimiter='$').astype('float64')[:,:] # Only load first 400 entries
#X_data /= np.max(X_data)    # Divide by the maximum WHY??

Y_data = np.loadtxt('./data_genotypes1.csv', delimiter='$').astype('float64')[:,:]

In [3]:
assert(len(X_data) == len(Y_data))

N = X_data.shape[0]
k = X_data.shape[1]
print("Dim. of X-data: %i x %i" % (N,k))
M1 = Y_data.shape[0]
M = Y_data.shape[1]
Y_data[Y_data < 0.5] = -1.0
print("Dim. of Y-data: %i x %i" % (M1,M))

Dim. of X-data: 961 x 2
Dim. of Y-data: 961 x 100


In [4]:
def py_func(func, inp, Tout, stateful=True, name=None, grad=None):
    
    # Need to generate a unique name to avoid duplicates:
    rnd_name = 'PyFuncGrad' + str(np.random.randint(0, 1E+8))
    
    tf.RegisterGradient(rnd_name)(grad)
    g = tf.get_default_graph()
    with g.gradient_override_map({"PyFunc": rnd_name}):
        return tf.py_func(func, inp, Tout, stateful=stateful, name=name)
    
def py_logdet(x, name=None):
    
    with ops.op_scope([x], name, "Logdet") as name:
        logdet = py_func(lambda x: np.linalg.slogdet(x)[1],
                         [x],
                         [tf.float64],
                         name=name,
                         grad=_LogdetGrad)  # <-- here's the call to the gradient
    return logdet[0]

def _LogdetGrad(op, grad):
    x = op.inputs[0]
    return tf.matrix_inverse(x) * grad

In [5]:
with tf.device('/cpu:0'):
    X = tf.Variable(dtype=tf.float64, initial_value=X_data, trainable=False)
    Y = tf.Variable(dtype=tf.float64, initial_value=Y_data, trainable=False)
    F = tf.Variable(dtype=tf.float64, initial_value=np.random.normal(0.0, 0.1, (N, M)).astype('float64'),
                    trainable=True)

    kernel_params = tf.placeholder(shape=[2], dtype=tf.float64)
    l = kernel_params[0]
    a = kernel_params[1]
    
    eye=tf.eye(N)

    K = a * tf.exp(-tf.reduce_sum(((X[:, None] - X[None, :]) ** 2) / (2 * l **2), reduction_indices=[2])) + \
        0.00001 * tf.eye(N, dtype=tf.float64) # Calculate Matrix of Covariances
    
    #K = a * tf.exp(-tf.reduce_sum(((X[:, None] - X[None, :]) ** 2) / (2 * l), reduction_indices=[2])) + \
       # 0.0001 * eye                

    p = tf.nn.sigmoid(F)            # Calculate probabilities(assuming sigmoid link function)
    Kinv_F = tf.matrix_solve(K, F)  # Calculate K^(-1)*F
    
    data_fit = -tf.reduce_sum(tf.log(1 + tf.exp(-Y * F)), reduction_indices=[0]) # Calculate Data Fit: Check  
    prior_fit = -0.5 * tf.reduce_sum(F * Kinv_F, reduction_indices=[0])          # Calcualte 1st term of prior probability: Check
    
    logL = tf.reduce_sum(data_fit + prior_fit) # Modulo terms not depending on f: Check
    
    g =  (0.5 * (Y + 1) - p) - Kinv_F # Calculate Matrix for Gradients Check
    
    W = p * (1 - p)        # Calculate first part of Hessia. It is diagonal (but here diagonal in every column)
    lhs = -K[:, :, None] * W[None, :, :] - tf.eye(N, dtype=tf.float64)[:, :, None] # Check (HEADACHE)
    #lhs = -K[:, :, None] * W[None, :, :] - eye[:, :, None] 
    rhs = tf.matmul(K, g)
    
    update = tf.matrix_solve(tf.transpose(lhs, [2, 0, 1]), tf.transpose(rhs, [1, 0])[:, :, None])[:, :, 0]
    update = tf.transpose(update)
    
    opt_op = F.assign(F - update)
    
    #####
    
    B = (W[:, None, :] ** 0.5) * K[:, :, None] * (W[None, :, :] ** 0.5) + tf.eye(N, dtype=tf.float64)[:, :, None]  # Check
    #B = (W[:, None, :] ** 0.5) * K[:, :, None] * (W[None, :, :] ** 0.5) + eye[:, :, None] 
    
    logdet = tf.reduce_sum(py_logdet(tf.transpose(B, [2, 0, 1])))  # Factor of 2??
    
    margL = logL - 0.5 * logdet
    
    
    ##### Gradients and Hessian #######################3
    
    grad = tf.gradients(margL, kernel_params)
    hessian = tf.hessians(margL, kernel_params)

In [ ]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.01

In [ ]:
### Single Run
aa, ll = 0.1, 25

with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    r = sess.run([opt_op, update, logL, margL], {a: aa, l: ll})
    print(r[-1])    # Print marginal Likelihood

In [ ]:
a_list = 0.01 * 2 ** np.arange(10)   # 10
l_list = 1.0 * 2 ** np.arange(10)  # 10

print(a_list)
print(l_list)

a_list = np.logspace(-1.5, -0.5, 10)
l_list = np.logspace(1, 2, 10)
print(a_list)
print(l_list)

In [ ]:
res = []
j=0
with tf.Session(config=config) as sess:
    for aa in a_list:
        for ll in l_list:
            print("Doing run: %i" % j)
            j+=1
    
            sess.run(tf.global_variables_initializer())   # Changed to global_variables_initializer

            prev = None
            for i in range(20):
                r = sess.run([opt_op, update, logL, margL], {a: aa, l:ll})
                if prev and np.abs(prev - r[-1]) < 0.01:
                    break
                prev = r[-1]
            print("Tested a: %4f" % aa)
            print("Tested l: %4f" % ll)
            print("Likelihood: %4f" % r[-1])
            res.append(r[-1])

In [ ]:
surface = np.array(res).reshape((10, 10))
pylab.pcolormesh(l_list, a_list, surface)
pylab.xscale('log')
pylab.yscale('log')
pylab.xlabel('l')
pylab.ylabel('a')
pylab.colorbar()
pylab.plot(25, 0.1, 'ko', linewidth=5)

In [ ]:
pylab.figure()
levels = np.arange(max(res) - 30, max(res) + 1, 2)  # Every two likelihood units
ax=pylab.contourf(l_list, a_list, surface, alpha=0.9, levels=levels)

# plt.clabel(ax, inline=1, fontsize=10)
pylab.colorbar(ax, format="%i")
pylab.title("Log Likelihood Surface", fontsize=20)
pylab.xlabel("l", fontsize=20)
pylab.ylabel("a", fontsize=20)
pylab.xscale('log')
pylab.yscale('log')
pylab.plot(25, 0.1, 'ko', linewidth=5, label="True Value")
pylab.legend()
pylab.show()

In [ ]:
np.max(surface) / (N * np.log(2) * M)

In [ ]:
with tf.Session(config=config) as sess:
    #sess.run(tf.initialize_all_variables())
    sess.run(tf.global_variables_initializer())# Update by Harald
    
    for i in range(3):
        sess.run(opt_op, {a: 0.1, l:25})
    r = sess.run(F, {a: 0.1, l: 25})
    print(r)

In [ ]:
x, y, z = X_data[:, 0], X_data[:, 1], r[:, 1]
x=np.linspace(0, 1, 20)
print(len(x))

In [ ]:
pylab.pcolormesh(x, x, z.reshape(21, 21))
pylab.colorbar()

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
z.shape

In [ ]:
961 ** 0.5

In [ ]:
with tf.Session() as sess:
    r = sess.run(tf.diag(tf.ones(5)))

In [ ]:
r

In [ ]:
tf.__version__

In [ ]:
logdet

In [ ]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    r = sess.run(logdet, {a: 1.0, l: 1.0})

In [ ]:
len(r)